In [1]:
import json

import pandas as pd
from langchain_core.documents import Document
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

from atomic_sovus.settings import EMBED_MODEL_ID

In [2]:
embedding_model = HuggingFaceEmbeddings(
    model_name=EMBED_MODEL_ID,
    model_kwargs={"device": "cpu"}
)

/home/stepan/.cache/pypoetry/virtualenvs/atomic-sovus-1llw-em--py3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/stepan/.cache/pypoetry/virtualenvs/atomic-sovus-1llw-em--py3.11/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


## Manual

In [2]:
with open('titles_all.json') as f:
    titles = json.load(f)

In [3]:
documents = []
for filename in titles.keys():
    docname = titles[filename]["name"]
    for titlename, page in titles[filename]["titles"].items():
        page_content = "\n".join([docname, titlename])
        doc = Document(
            page_content=page_content,
            metadata={"filename": filename, "page": page, "docname": docname, "titlename": titlename}
        )
        documents.append(doc)

In [9]:
faiss_index_manual = FAISS.from_documents(documents, embedding_model)

In [ ]:
faiss_index_manual.save_local("../vectorstores/manual/faiss_index")

## FAQ

In [ ]:
faq_df = pd.read_excel("../../data/Вопросы и ответы.xlsx", sheet_name="TDSheet", header=1)[["Описание", "Решение"]]

In [ ]:
documents_faq = []
for _, row in faq_df.iterrows():
    question_from_faq = row["Описание"]
    answer_from_faq = row["Решение"]
    doc_faq = Document(
        page_content=question_from_faq,
        metadata={"filename": "Вопросы и ответы.xlsx", "answer": answer_from_faq},
    )
    documents_faq.append(doc_faq)

In [ ]:
faiss_index_faq = FAISS.from_documents(documents_faq, embedding_model)

In [ ]:
faiss_index_faq.save_local("../vectorstores/faq/faiss_index")

## Tests

In [3]:
faiss_index_manual = FAISS.load_local("../vectorstores/manual/faiss_index", embedding_model, allow_dangerous_deserialization=True)
retriever_manual = faiss_index_manual.as_retriever(search_kwargs={"k": 5})

faiss_index_faq = FAISS.load_local("../vectorstores/faq/faiss_index", embedding_model, allow_dangerous_deserialization=True)
retriever_faq = faiss_index_faq.as_retriever(search_kwargs={"k": 3})

In [23]:
question = "Как списать аванс по безналу?"

## 1 сито

In [24]:
retrieved_docs = retriever_manual.invoke(question)

In [25]:
print(f"Вот что удалось найти по вопросу\n\"{question}\"\n\n")
for i, retrieved_doc in enumerate(retrieved_docs):
    filename = retrieved_doc.metadata["filename"]
    titlename = retrieved_doc.metadata["titlename"]
    page = retrieved_doc.metadata["page"]
    # print(retrieved_doc.page_content)
    print(f"файл: {filename};\nпункт: {titlename};\nстр. {page}")
    print("\n" + "="*30 + "\n")

Вот что удалось найти по вопросу
"Как списать аванс по безналу?"


файл: Инструкция D-1C1-1.10.04 Учет банковских операций (НФ)_v2 (28) (1).pdf;
пункт: 6.3 «Поступление безналичных ДС»). Высвобождение аванса.;
стр. 137


файл: Инструкция D-1C1-1.10.04 Учет банковских операций (НФ)_v2 (28) (1).pdf;
пункт: 6.2 Возврат аванса, где объектом расчетов является сам документ «Списание безналичных ДС»;
стр. 130


файл: Инструкция пользователей «D-1C1-1.22.01 Формирование графиков платежей, поступления ДС, поставок, отгрузок».pdf;
пункт: Заполнение дат планового погашения авансов в документах «Списание безналичных ДС», «Поступление безналичных ДС»;
стр. 22


файл: Инструкция D-1C1-1.10.04 Учет банковских операций (НФ)_v2 (28) (1).pdf;
пункт: 6.1 Возврат аванса от поставщика, где объект расчета Заказ поставщику;
стр. 124


файл: Инструкция D-1C1-1.10.04 Учет банковских операций (НФ)_v2 (28) (1).pdf;
пункт: Бухгалтер по учету банковских операций: Проверка документа «Списание безналичных денежных с

## 2 сито

In [26]:
retrieved_docs = retriever_faq.invoke(question)

In [27]:
print(f"Вот что удалось найти по истории сообщений\n\"{question}\"\n\n")
for i, retrieved_doc in enumerate(retrieved_docs):
    _question = retrieved_doc.page_content
    answer = retrieved_doc.metadata["answer"]
    print(f"вопрос: {_question};\nответ: {answer}")
    print("\n" + "="*30 + "\n")

Вот что удалось найти по истории сообщений
"Как списать аванс по безналу?"


вопрос:  Здравствуйте.  Через авансовый отчет проходят услуги, по которым НДС надо списать на непринимаемую статью затрат сч.XX.XX.X Невозмещаемый НДС не приним.  Каким образом это сделать?;
ответ: Добрый день Елена Всеволодовна! В копии базы проведен АО XX от XX.XX.XXXX, сформированы проводки по которым затраты отнесены на Принимаемую статью, а ндс на Не принимаемую. Закрытие обращения согласовано с пользователем.


вопрос: Добрый день! Коллеги, подскажите пожалуйста как заводить авансовую счет фактуру ;
ответ: Добрый день Наталья Александровна! Счета-фактуры полученные на аванс (XX.АВ),  в соответствии с целевым отчетным процессом, формируются сотрудниками ЦО.


вопрос: Документ не формирует авансовую проводку. Почему?;
ответ: Корректировка задолженности по фин. Устанавливаем корректировку кредиторской задолженности. Необходимо установить   флаг изменить курс взаиморасчетов, тогда проводки сформируются. Аван